In [ ]:
# Cell 1: Install Dependencies
# ==================================================================================
!pip install segmentation_models_pytorch wandb -q

In [ ]:
# Cell 2: Import Libraries
# ==================================================================================
import os
import cv2
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import random
import warnings
warnings.filterwarnings('ignore')

# PyTorch imports
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import OneCycleLR
from torch.cuda.amp import GradScaler, autocast

# Model, Augmentations, and Utilities
import segmentation_models_pytorch as smp
import albumentations as A
from albumentations.pytorch import ToTensorV2
from tqdm.notebook import tqdm
from sklearn.model_selection import KFold, train_test_split

# Experiment Tracking
import wandb

print("All libraries imported successfully!")


In [ ]:
# Cell 3: W&B Login
# ==================================================================================
from kaggle_secrets import UserSecretsClient
try:
    user_secrets = UserSecretsClient()
    wandb_api_key = user_secrets.get_secret("wandb_api")
    wandb.login(key=wandb_api_key)
    print("✓ Logged in to W&B successfully!")
except:
    wandb.login(anonymous='must')
    print("⚠ Using anonymous W&B login")

In [ ]:
# Cell 4: Seeding for Reproducibility
# ==================================================================================
def seed_everything(seed=42):
    """Set seeds for reproducibility"""
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(42)
print("✓ Random seeds set for reproducibility")


In [ ]:
# Cell 5: Configuration
# ==================================================================================
config = {
    # ========== DATA PATHS ==========
    # For baseline experiment, you only need the gold standard dataset
    "GOLD_STANDARD_PATH": '/kaggle/input/200-gold-standard-adni/200_AD_CN_MCI_11112025/',
    "GOLD_METADATA": '/kaggle/input/metadata/metadata.csv',
    
    # Pseudo-label paths (not used in baseline, but keep for later experiments)
    "PSEUDO_LABEL_PATH": '/kaggle/input/bronze-new-standard-5300/PSEUDO_LABELS/',
    "PSEUDO_METADATA": '/kaggle/input/bronze-new-standard-5300/PSEUDO_LABELS/metadata.csv',
    
    # ========== IMAGE SETTINGS ==========
    "IMG_HEIGHT": 320,
    "IMG_WIDTH": 320,
    
    # ========== MODEL ARCHITECTURE ==========
    "MODEL_ARCHITECTURE": "UnetPlusPlus",
    "ENCODER": 'efficientnet-b4',
    "PRETRAINED_WEIGHTS": 'imagenet',
    
    # ========== TRAINING HYPERPARAMETERS ==========
    "BATCH_SIZE": 32,
    "EPOCHS": 150,
    "LEARNING_RATE": 1e-4,
    "WEIGHT_DECAY": 1e-5,
    "DEVICE": "cuda" if torch.cuda.is_available() else "cpu",
    "NUM_WORKERS": 4,
    
    # ========== TRAINING STRATEGY ==========
    "PATIENCE": 40,
    "USE_PSEUDO_LABELS": True,  # ⬅️ START WITH FALSE - Baseline experiment first!
    
    # ========== CROSS-VALIDATION ==========
    "N_FOLDS": 5,
    "CURRENT_FOLD": None,  # Will be set during training
    
    # ========== W&B CONFIGURATION ==========
    "WANDB_PROJECT": "Brain-Segmentation-eXP3",  # Changed for baseline experiment
    "ENTITY": "alzhemer_segmentaion",
    "RUN_NAME": None,  # Will be set per fold
}

print(f"✓ Configuration loaded")
print(f"✓ Using device: {config['DEVICE']}")
print(f"✓ GPU Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"✓ GPU Name: {torch.cuda.get_device_name(0)}")

# ========== EXPERIMENT INFORMATION ==========
print("\n" + "="*80)
print("🧪 EXPERIMENT: BASELINE (SUPERVISED LEARNING)")
print("="*80)
print(f"Training Strategy:")
print(f"  • Training:   ~160 gold standard images per fold")
print(f"  • Validation: ~40 gold standard images per fold")
print(f"  • Pseudo-labels: NOT USED (establishing baseline)")
print(f"  • Purpose: Establish baseline performance for comparison")
print(f"  • Expected Dice: ~0.88-0.92")
print("="*80)


In [ ]:
# Cell 6: Load and Prepare Data
# ==================================================================================
print("\n" + "="*80)
print("LOADING DATA")
print("="*80)

# Load gold standard data (200 manually labeled images)
gold_metadata = pd.read_csv(config['GOLD_METADATA'])
print(f"\n✓ Gold Standard Images: {len(gold_metadata)}")

# Load pseudo-labeled data (5000 images)
if config['USE_PSEUDO_LABELS']:
    pseudo_metadata = pd.read_csv(config['PSEUDO_METADATA'])
    print(f"✓ Pseudo-Labeled Images: {len(pseudo_metadata)}")
    print(f"✓ Total Training Pool: {len(gold_metadata) + len(pseudo_metadata)}")
else:
    pseudo_metadata = None
    print("⚠ Training ONLY on Gold Standard (no pseudo-labels)")

# Add source identifier
gold_metadata['source'] = 'gold'
if pseudo_metadata is not None:
    pseudo_metadata['source'] = 'pseudo'

In [ ]:
# Cell 7: Data Augmentation Pipeline
# ==================================================================================
print("\n" + "="*80)
print("AUGMENTATION PIPELINE")
print("="*80)

# Training augmentations - clinically informed
train_augs = A.Compose([
    A.PadIfNeeded(
        min_height=config['IMG_HEIGHT'],
        min_width=config['IMG_WIDTH'],
        border_mode=cv2.BORDER_CONSTANT,
    ),
    A.Resize(height=config['IMG_HEIGHT'], width=config['IMG_WIDTH']),
    
    # Geometric transformations (simulate patient positioning)
    A.Affine(
        scale=(0.95, 1.05),
        translate_percent=(-0.05, 0.05),
        rotate=(-7, 7),
        p=0.8
    ),
    
    # Intensity augmentations (simulate scanner variations)
    A.RandomBrightnessContrast(p=0.4),
    A.GaussNoise(p=0.3),
    A.CLAHE(p=0.5),
    A.Sharpen(p=0.3),
    
    # Normalization and conversion
    A.Normalize(mean=(0.5,), std=(0.5,)),
    ToTensorV2(),
])

# Validation augmentations - no augmentation, only preprocessing
val_augs = A.Compose([
    A.PadIfNeeded(
        min_height=config['IMG_HEIGHT'],
        min_width=config['IMG_WIDTH'],
        border_mode=cv2.BORDER_CONSTANT,
    ),
    A.Resize(height=config['IMG_HEIGHT'], width=config['IMG_WIDTH']),
    A.Normalize(mean=(0.5,), std=(0.5,)),
    ToTensorV2(),
])

print("✓ Augmentation pipelines created")

In [ ]:
# Cell 8: Custom Dataset Class
# ==================================================================================
class BrainMRIDataset(Dataset):
    """
    Custom Dataset for Brain MRI Segmentation
    Handles both gold standard and pseudo-labeled data
    """
    def __init__(self, df, base_path, augmentations=None):
        self.df = df.reset_index(drop=True)
        self.base_path = base_path
        self.augmentations = augmentations
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        
        # Construct paths based on source
        if row['source'] == 'gold':
            image_path = os.path.join(self.base_path['gold'], 'images', row['image_id'])
            mask_path = os.path.join(self.base_path['gold'], 'masks', row['mask_id'])
        else:
            image_path = os.path.join(self.base_path['pseudo'], 'images965', row['image_id'])
            mask_path = os.path.join(self.base_path['pseudo'], 'masks965', row['mask_id'])
        
        # Read image and mask
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        
        # Ensure shape consistency
        if image.shape != mask.shape:
            mask = cv2.resize(mask, (image.shape[1], image.shape[0]), 
                            interpolation=cv2.INTER_NEAREST)
        
        # Add channel dimension
        image = np.expand_dims(image, axis=-1)
        mask = np.expand_dims(mask, axis=-1)
        
        # Apply augmentations
        if self.augmentations:
            transformed = self.augmentations(image=image, mask=mask)
            image = transformed['image']
            mask = transformed['mask']
        
        # Convert mask to binary
        mask = (mask / 255.0 > 0.5).float()
        
        return image, mask.permute(2, 0, 1)

print("✓ Custom Dataset class defined")

In [ ]:
# Cell 9: Loss Functions and Metrics
# ==================================================================================
class TverskyLoss(nn.Module):
    """Tversky Loss - handles class imbalance"""
    def __init__(self, alpha=0.7, beta=0.3, smooth=1e-6):
        super(TverskyLoss, self).__init__()
        self.alpha = alpha
        self.beta = beta
        self.smooth = smooth
        
    def forward(self, inputs, targets):
        inputs = torch.sigmoid(inputs)
        inputs, targets = inputs.view(-1), targets.view(-1)
        
        true_pos = (inputs * targets).sum()
        false_neg = ((1 - inputs) * targets).sum()
        false_pos = (inputs * (1 - targets)).sum()
        
        tversky_index = (true_pos + self.smooth) / (
            true_pos + self.alpha * false_neg + self.beta * false_pos + self.smooth
        )
        return 1 - tversky_index

class FocalLoss(nn.Module):
    """Focal Loss - focuses on hard examples"""
    def __init__(self, alpha=0.8, gamma=3.0, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction
        
    def forward(self, inputs, targets):
        bce_loss = nn.BCEWithLogitsLoss(reduction='none')(inputs, targets)
        pt = torch.exp(-bce_loss)
        focal_loss = self.alpha * (1 - pt)**self.gamma * bce_loss
        return focal_loss.mean() if self.reduction == 'mean' else focal_loss

class WeightedFocalTverskyLoss(nn.Module):
    """Combined Focal + Tversky Loss"""
    def __init__(self, focal_weight=0.6, tversky_weight=0.4):
        super(WeightedFocalTverskyLoss, self).__init__()
        self.focal_loss = FocalLoss()
        self.tversky_loss = TverskyLoss()
        self.focal_weight = focal_weight
        self.tversky_weight = tversky_weight
        
    def forward(self, inputs, targets):
        return (self.focal_weight * self.focal_loss(inputs, targets) + 
                self.tversky_weight * self.tversky_loss(inputs, targets))

class BCEFocalTverskyLoss(nn.Module):
    """Ultimate Combined Loss: BCE + Focal + Tversky"""
    def __init__(self, bce_weight=0.3, focal_tversky_weight=0.7):
        super(BCEFocalTverskyLoss, self).__init__()
        self.bce_loss = nn.BCEWithLogitsLoss()
        self.focal_tversky_loss = WeightedFocalTverskyLoss()
        self.bce_weight = bce_weight
        self.focal_tversky_weight = focal_tversky_weight
        
    def forward(self, inputs, targets):
        bce = self.bce_loss(inputs, targets)
        focal_tversky = self.focal_tversky_loss(inputs, targets)
        return self.bce_weight * bce + self.focal_tversky_weight * focal_tversky

def dice_coef(y_pred, y_true, smooth=1):
    """Dice Coefficient - primary evaluation metric"""
    y_pred_sig = torch.sigmoid(y_pred)
    y_pred_bin = (y_pred_sig > 0.5).float()
    
    intersection = (y_pred_bin.view(-1) * y_true.view(-1)).sum()
    return (2. * intersection + smooth) / (y_pred_bin.sum() + y_true.sum() + smooth)

def iou_score(y_pred, y_true, smooth=1):
    """IoU Score - additional metric"""
    y_pred_sig = torch.sigmoid(y_pred)
    y_pred_bin = (y_pred_sig > 0.5).float()
    
    intersection = (y_pred_bin.view(-1) * y_true.view(-1)).sum()
    union = y_pred_bin.sum() + y_true.sum() - intersection
    return (intersection + smooth) / (union + smooth)

print("✓ Loss functions and metrics defined")


In [ ]:
 #Cell 10: Model Creation Function
# ==================================================================================
def create_model(config):
    """Create U-Net++ model with specified encoder"""
    model = smp.UnetPlusPlus(
        encoder_name=config['ENCODER'],
        encoder_weights=config['PRETRAINED_WEIGHTS'],
        in_channels=1,
        classes=1
    ).to(config['DEVICE'])
    
    # Multi-GPU support
    if torch.cuda.device_count() > 1:
        print(f"✓ Using {torch.cuda.device_count()} GPUs!")
        model = nn.DataParallel(model)
    
    return model

print("✓ Model creation function defined")

In [ ]:
# Cell 11: Training Function
# ==================================================================================
def train_one_epoch(model, train_loader, optimizer, loss_fn, scaler, scheduler, device):
    """Train for one epoch"""
    model.train()
    total_loss = 0.0
    total_dice = 0.0
    total_iou = 0.0
    
    loop = tqdm(train_loader, desc="Training", leave=False)
    
    for images, masks in loop:
        images = images.to(device)
        masks = masks.to(device)
        
        optimizer.zero_grad(set_to_none=True)
        
        with autocast():
            outputs = model(images)
            loss = loss_fn(outputs, masks)
        
        scaler.scale(loss).backward()
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        scaler.step(optimizer)
        scaler.update()
        scheduler.step()
        
        total_loss += loss.item()
        total_dice += dice_coef(outputs, masks).item()
        total_iou += iou_score(outputs, masks).item()
        
        loop.set_postfix({
            "loss": f"{loss.item():.4f}",
            "dice": f"{dice_coef(outputs, masks).item():.4f}",
            "lr": f"{scheduler.get_last_lr()[0]:.2e}"
        })
    
    return {
        'loss': total_loss / len(train_loader),
        'dice': total_dice / len(train_loader),
        'iou': total_iou / len(train_loader)
    }

def validate(model, val_loader, loss_fn, device):
    """Validate the model"""
    model.eval()
    total_loss = 0.0
    total_dice = 0.0
    total_iou = 0.0
    
    with torch.no_grad():
        loop = tqdm(val_loader, desc="Validation", leave=False)
        
        for images, masks in loop:
            images = images.to(device)
            masks = masks.to(device)
            
            with autocast():
                outputs = model(images)
                loss = loss_fn(outputs, masks)
            
            total_loss += loss.item()
            total_dice += dice_coef(outputs, masks).item()
            total_iou += iou_score(outputs, masks).item()
            
            loop.set_postfix({
                "loss": f"{loss.item():.4f}",
                "dice": f"{dice_coef(outputs, masks).item():.4f}"
            })
    
    return {
        'loss': total_loss / len(val_loader),
        'dice': total_dice / len(val_loader),
        'iou': total_iou / len(val_loader)
    }

print("✓ Training and validation functions defined")

In [ ]:
# Cell 12: K-Fold Cross-Validation Training
# ==================================================================================
print("\n" + "="*80)
print("STARTING K-FOLD CROSS-VALIDATION")
print("="*80)

# Prepare K-Fold splits on gold standard data ONLY
kfold = KFold(n_splits=config['N_FOLDS'], shuffle=True, random_state=42)
fold_results = []

# Base paths for datasets
base_path = {
    'gold': config['GOLD_STANDARD_PATH'],
    'pseudo': config['PSEUDO_LABEL_PATH']
}

for fold, (train_idx, val_idx) in enumerate(kfold.split(gold_metadata), 1):
    print(f"\n{'='*80}")
    print(f"FOLD {fold}/{config['N_FOLDS']}")
    print(f"{'='*80}")
    
    config['CURRENT_FOLD'] = fold
    config['RUN_NAME'] = f"Fold{fold}_EfficientNetB4_320px_SemiSupervised"
    
    # Initialize W&B for this fold
    wandb.init(
        project=config['WANDB_PROJECT'],
        entity=config['ENTITY'],
        config=config,
        name=config['RUN_NAME'],
        reinit=True
    )
    
    # Split gold standard data
    train_gold = gold_metadata.iloc[train_idx].copy()
    val_gold = gold_metadata.iloc[val_idx].copy()
    
    print(f"\n📊 Data Split:")
    print(f"  Gold Standard Training: {len(train_gold)}")
    print(f"  Gold Standard Validation: {len(val_gold)}")
    
    # Combine with pseudo-labeled data for training
    if config['USE_PSEUDO_LABELS']:
        train_combined = pd.concat([train_gold, pseudo_metadata], ignore_index=True)
        print(f"  Pseudo-Labeled Training: {len(pseudo_metadata)}")
        print(f"  Total Training: {len(train_combined)}")
    else:
        train_combined = train_gold
        print(f"  Total Training: {len(train_combined)}")
    
    # Create datasets and dataloaders
    train_dataset = BrainMRIDataset(train_combined, base_path, augmentations=train_augs)
    val_dataset = BrainMRIDataset(val_gold, base_path, augmentations=val_augs)
    
    train_loader = DataLoader(
        train_dataset,
        batch_size=config['BATCH_SIZE'],
        shuffle=True,
        num_workers=config['NUM_WORKERS'],
        pin_memory=True
    )
    
    val_loader = DataLoader(
        val_dataset,
        batch_size=config['BATCH_SIZE'],
        shuffle=False,
        num_workers=config['NUM_WORKERS'],
        pin_memory=True
    )
    
    # Create model, optimizer, scheduler, loss
    model = create_model(config)
    loss_fn = BCEFocalTverskyLoss()
    
    optimizer = torch.optim.AdamW(
        model.parameters(),
        lr=config['LEARNING_RATE'],
        weight_decay=config['WEIGHT_DECAY']
    )
    
    scheduler = OneCycleLR(
        optimizer,
        max_lr=config['LEARNING_RATE'],
        epochs=config['EPOCHS'],
        steps_per_epoch=len(train_loader),
        pct_start=0.1,
        div_factor=10,
        final_div_factor=100
    )
    
    scaler = GradScaler()
    
    # Training loop
    best_val_dice = 0.0
    patience_counter = 0
    history = {
        'train_loss': [], 'val_loss': [],
        'train_dice': [], 'val_dice': [],
        'train_iou': [], 'val_iou': []
    }
    
    print(f"\n🚀 Starting Training for Fold {fold}...")
    
    for epoch in range(config['EPOCHS']):
        print(f"\nEpoch {epoch+1}/{config['EPOCHS']}")
        
        # Train
        train_metrics = train_one_epoch(
            model, train_loader, optimizer, loss_fn, scaler, scheduler, config['DEVICE']
        )
        
        # Validate
        val_metrics = validate(model, val_loader, loss_fn, config['DEVICE'])
        
        # Store history
        history['train_loss'].append(train_metrics['loss'])
        history['train_dice'].append(train_metrics['dice'])
        history['train_iou'].append(train_metrics['iou'])
        history['val_loss'].append(val_metrics['loss'])
        history['val_dice'].append(val_metrics['dice'])
        history['val_iou'].append(val_metrics['iou'])
        
        # Log to W&B
        wandb.log({
            "fold": fold,
            "epoch": epoch + 1,
            "train/loss": train_metrics['loss'],
            "train/dice": train_metrics['dice'],
            "train/iou": train_metrics['iou'],
            "val/loss": val_metrics['loss'],
            "val/dice": val_metrics['dice'],
            "val/iou": val_metrics['iou'],
            "learning_rate": scheduler.get_last_lr()[0]
        })
        
        # Print epoch summary
        print(f"  Train - Loss: {train_metrics['loss']:.4f}, "
              f"Dice: {train_metrics['dice']:.4f}, IoU: {train_metrics['iou']:.4f}")
        print(f"  Val   - Loss: {val_metrics['loss']:.4f}, "
              f"Dice: {val_metrics['dice']:.4f}, IoU: {val_metrics['iou']:.4f}")
        
        # Log predictions every 10 epochs
        if (epoch + 1) % 10 == 0:
            sample_images, sample_masks = next(iter(val_loader))
            sample_images = sample_images.to(config['DEVICE'])
            sample_masks = sample_masks.to(config['DEVICE'])
            
            model.eval()
            with torch.no_grad(), autocast():
                preds_raw = model(sample_images)
                probs = torch.sigmoid(preds_raw)
                preds = (probs > 0.5).float()
            
            log_images = []
            for i in range(min(4, config["BATCH_SIZE"])):
                comparison_img = torch.cat([
                    sample_images[i],
                    sample_masks[i],
                    preds[i]
                ], dim=2)
                
                log_images.append(
                    wandb.Image(
                        comparison_img,
                        caption=f"Fold {fold}, Epoch {epoch+1}: Input | GT | Prediction"
                    )
                )
            
            wandb.log({"val/predictions": log_images}, commit=False)
        
        # Save best model
        if val_metrics['dice'] > best_val_dice:
            best_val_dice = val_metrics['dice']
            model_state = model.module.state_dict() if isinstance(model, nn.DataParallel) else model.state_dict()
            torch.save(model_state, f"best_model_fold{fold}.pth")
            print(f"  ✓ Model saved! Best Val Dice: {best_val_dice:.4f}")
            patience_counter = 0
        else:
            patience_counter += 1
            
        # Early stopping
        if patience_counter >= config['PATIENCE']:
            print(f"\n⚠ Early stopping triggered after {patience_counter} epochs without improvement")
            break
    
    # Store fold results
    fold_results.append({
        'fold': fold,
        'best_val_dice': best_val_dice,
        'best_val_loss': min(history['val_loss']),
        'final_train_dice': history['train_dice'][-1],
        'final_val_dice': history['val_dice'][-1]
    })
    
    # Log final fold summary
    wandb.log({
        f"fold{fold}/best_val_dice": best_val_dice,
        f"fold{fold}/best_val_loss": min(history['val_loss'])
    })
    
    # Finish W&B run for this fold
    wandb.finish()
    
    print(f"\n✓ Fold {fold} Complete - Best Val Dice: {best_val_dice:.4f}")


In [ ]:
# Cell 13: Cross-Validation Results Summary
# ==================================================================================
print("\n" + "="*80)
print("K-FOLD CROSS-VALIDATION RESULTS")
print("="*80)

results_df = pd.DataFrame(fold_results)
print("\n📊 Individual Fold Results:")
print(results_df.to_string(index=False))

print(f"\n📈 Summary Statistics:")
print(f"  Mean Val Dice: {results_df['best_val_dice'].mean():.4f} ± {results_df['best_val_dice'].std():.4f}")
print(f"  Min Val Dice:  {results_df['best_val_dice'].min():.4f}")
print(f"  Max Val Dice:  {results_df['best_val_dice'].max():.4f}")

# Save results
results_df.to_csv('kfold_cv_results.csv', index=False)
print(f"\n✓ Results saved to 'kfold_cv_results.csv'")

In [ ]:
# Cell 14: Visualization - Training Curves
# ==================================================================================
print("\n" + "="*80)
print("VISUALIZING TRAINING CURVES")
print("="*80)

# Note: This will show the last fold's training curves
# For production, you'd want to save and plot all folds

plt.style.use('seaborn-v0_8-darkgrid')
fig, axes = plt.subplots(2, 2, figsize=(18, 12))

epochs_ran = range(1, len(history['train_loss']) + 1)

# Plot 1: Loss
axes[0, 0].plot(epochs_ran, history['train_loss'], 'b-o', label='Training Loss', alpha=0.7)
axes[0, 0].plot(epochs_ran, history['val_loss'], 'r-o', label='Validation Loss', alpha=0.7)
axes[0, 0].set_title('Training & Validation Loss', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('Epochs', fontsize=12)
axes[0, 0].set_ylabel('Loss', fontsize=12)
axes[0, 0].legend(fontsize=11)
axes[0, 0].grid(True, alpha=0.3)

# Plot 2: Dice Coefficient
axes[0, 1].plot(epochs_ran, history['train_dice'], 'b-o', label='Training Dice', alpha=0.7)
axes[0, 1].plot(epochs_ran, history['val_dice'], 'r-o', label='Validation Dice', alpha=0.7)
axes[0, 1].set_title('Training & Validation Dice Coefficient', fontsize=14, fontweight='bold')
axes[0, 1].set_xlabel('Epochs', fontsize=12)
axes[0, 1].set_ylabel('Dice Coefficient', fontsize=12)
axes[0, 1].legend(fontsize=11)
axes[0, 1].grid(True, alpha=0.3)

# Plot 3: IoU Score
axes[1, 0].plot(epochs_ran, history['train_iou'], 'b-o', label='Training IoU', alpha=0.7)
axes[1, 0].plot(epochs_ran, history['val_iou'], 'r-o', label='Validation IoU', alpha=0.7)
axes[1, 0].set_title('Training & Validation IoU Score', fontsize=14, fontweight='bold')
axes[1, 0].set_xlabel('Epochs', fontsize=12)
axes[1, 0].set_ylabel('IoU Score', fontsize=12)
axes[1, 0].legend(fontsize=11)
axes[1, 0].grid(True, alpha=0.3)

# Plot 4: Cross-Validation Summary
fold_numbers = results_df['fold'].values
fold_dice_scores = results_df['best_val_dice'].values
axes[1, 1].bar(fold_numbers, fold_dice_scores, alpha=0.7, color='steelblue', edgecolor='black')
axes[1, 1].axhline(y=fold_dice_scores.mean(), color='red', linestyle='--', 
                   label=f'Mean: {fold_dice_scores.mean():.4f}', linewidth=2)
axes[1, 1].set_title('Best Validation Dice per Fold', fontsize=14, fontweight='bold')
axes[1, 1].set_xlabel('Fold', fontsize=12)
axes[1, 1].set_ylabel('Dice Coefficient', fontsize=12)
axes[1, 1].set_xticks(fold_numbers)
axes[1, 1].legend(fontsize=11)
axes[1, 1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('training_curves.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Training curves saved as 'training_curves.png'")

In [ ]:
# Cell 15: Test Best Model from Best Fold
# ==================================================================================
print("\n" + "="*80)
print("TESTING BEST MODEL")
print("="*80)

# Find best fold
best_fold_idx = results_df['best_val_dice'].idxmax()
best_fold = results_df.loc[best_fold_idx, 'fold']
best_dice = results_df.loc[best_fold_idx, 'best_val_dice']

print(f"\n🏆 Best Fold: {best_fold} (Val Dice: {best_dice:.4f})")

# Load best model
print(f"\n📂 Loading model from Fold {best_fold}...")
best_model = create_model(config)
best_model_state = torch.load(f"best_model_fold{best_fold}.pth")
if isinstance(best_model, nn.DataParallel):
    best_model.module.load_state_dict(best_model_state)
else:
    best_model.load_state_dict(best_model_state)
best_model.eval()
print("✓ Model loaded successfully")

# Create test set from validation fold of best model
_, test_idx = list(kfold.split(gold_metadata))[best_fold - 1]
test_gold = gold_metadata.iloc[test_idx].copy()

print(f"\n📊 Test Set: {len(test_gold)} images (from Fold {best_fold} validation)")

test_dataset = BrainMRIDataset(test_gold, base_path, augmentations=val_augs)
test_loader = DataLoader(
    test_dataset,
    batch_size=config['BATCH_SIZE'],
    shuffle=False,
    num_workers=config['NUM_WORKERS'],
    pin_memory=True
)

# Test with TTA (Test-Time Augmentation)
print("\n🧪 Testing with Test-Time Augmentation...")
test_loss = 0.0
test_dice = 0.0
test_iou = 0.0
loss_fn = BCEFocalTverskyLoss()

with torch.no_grad():
    loop = tqdm(test_loader, desc="Testing")
    for images, masks in loop:
        images = images.to(config['DEVICE'])
        masks = masks.to(config['DEVICE'])
        
        with autocast():
            # Original prediction
            outputs_original = best_model(images)
            
            # Flipped prediction
            flipped_images = torch.flip(images, [3])
            outputs_flipped = best_model(flipped_images)
            outputs_flipped = torch.flip(outputs_flipped, [3])
            
            # Average predictions (TTA)
            outputs = (outputs_original + outputs_flipped) / 2.0
        
        loss = loss_fn(outputs, masks)
        test_loss += loss.item()
        test_dice += dice_coef(outputs, masks).item()
        test_iou += iou_score(outputs, masks).item()
        
        loop.set_postfix({
            "loss": f"{loss.item():.4f}",
            "dice": f"{dice_coef(outputs, masks).item():.4f}"
        })

avg_test_loss = test_loss / len(test_loader)
avg_test_dice = test_dice / len(test_loader)
avg_test_iou = test_iou / len(test_loader)

print("\n" + "="*80)
print("FINAL TEST RESULTS (with TTA)")
print("="*80)
print(f"  Test Loss: {avg_test_loss:.4f}")
print(f"  Test Dice: {avg_test_dice:.4f}")
print(f"  Test IoU:  {avg_test_iou:.4f}")
print("="*80)


In [ ]:
# Cell 16: Visualize Sample Predictions
# ==================================================================================
print("\n" + "="*80)
print("VISUALIZING SAMPLE PREDICTIONS")
print("="*80)

num_samples = 10
indices = np.random.choice(range(len(test_dataset)), 
                          min(num_samples, len(test_dataset)), 
                          replace=False)

best_model.eval()
fig, axes = plt.subplots(num_samples, 4, figsize=(20, num_samples*5))

with torch.no_grad():
    for plot_idx, data_idx in enumerate(indices):
        img_tensor, gt_tensor = test_dataset[data_idx]
        img_input = img_tensor.unsqueeze(0).to(config['DEVICE'])
        
        # TTA prediction
        with autocast():
            outputs_original = torch.sigmoid(best_model(img_input))
            flipped_images = torch.flip(img_input, [3])
            outputs_flipped = torch.sigmoid(best_model(flipped_images))
            outputs_flipped = torch.flip(outputs_flipped, [3])
            final_prob = (outputs_original + outputs_flipped) / 2.0
        
        pred_mask = (final_prob > 0.5).cpu().numpy().squeeze()
        img_np = img_tensor.numpy().squeeze()
        gt_np = gt_tensor.numpy().squeeze()
        
        # Overlay visualization
        overlay = cv2.cvtColor((img_np * 255).astype(np.uint8), cv2.COLOR_GRAY2RGB)
        overlay[pred_mask > 0.5] = [255, 0, 0]  # Red for predictions
        overlay[gt_np > 0.5] = [0, 255, 0]  # Green for ground truth
        
        # Calculate metrics for this sample
        sample_dice = dice_coef(
            torch.from_numpy(pred_mask).unsqueeze(0).unsqueeze(0),
            torch.from_numpy(gt_np).unsqueeze(0).unsqueeze(0)
        ).item()
        
        # Plot
        if num_samples == 1:
            ax = [axes]
        else:
            ax = axes[plot_idx]
        
        ax[0].imshow(img_np, cmap='gray')
        ax[0].set_title(f'Input Image {plot_idx+1}', fontsize=12)
        ax[0].axis('off')
        
        ax[1].imshow(gt_np, cmap='gray')
        ax[1].set_title('Ground Truth', fontsize=12)
        ax[1].axis('off')
        
        ax[2].imshow(pred_mask, cmap='gray')
        ax[2].set_title(f'Prediction (Dice: {sample_dice:.3f})', fontsize=12)
        ax[2].axis('off')
        
        ax[3].imshow(overlay)
        ax[3].set_title('Overlay (Red=Pred, Green=GT)', fontsize=12)
        ax[3].axis('off')

plt.tight_layout()
plt.savefig('sample_predictions.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Sample predictions saved as 'sample_predictions.png'")


In [ ]:
# Cell 17: Final Summary Report
# ==================================================================================
print("\n" + "="*80)
print("FINAL SUMMARY REPORT")
print("="*80)

summary_report = f"""
╔════════════════════════════════════════════════════════════════════════════╗
║                   BRAIN SEGMENTATION - FINAL REPORT                        ║
╚════════════════════════════════════════════════════════════════════════════╝

📊 DATASET INFORMATION:
   • Gold Standard (Manual Labels):  {len(gold_metadata)} images
   • Pseudo-Labels (Used in Training): {len(pseudo_metadata) if pseudo_metadata is not None else 0} images
   • Total Training Pool:              {len(gold_metadata) + (len(pseudo_metadata) if pseudo_metadata is not None else 0)} images

🏗️ MODEL ARCHITECTURE:
   • Architecture:  {config['MODEL_ARCHITECTURE']}
   • Encoder:       {config['ENCODER']}
   • Input Size:    {config['IMG_HEIGHT']}x{config['IMG_WIDTH']}
   • Loss Function: BCE + Focal + Tversky (Weighted Combination)

🔬 CROSS-VALIDATION RESULTS ({config['N_FOLDS']}-Fold CV on Gold Standard):
   • Mean Dice Score:  {results_df['best_val_dice'].mean():.4f} ± {results_df['best_val_dice'].std():.4f}
   • Min Dice Score:   {results_df['best_val_dice'].min():.4f} (Fold {results_df.loc[results_df['best_val_dice'].idxmin(), 'fold']})
   • Max Dice Score:   {results_df['best_val_dice'].max():.4f} (Fold {results_df.loc[results_df['best_val_dice'].idxmax(), 'fold']})

🏆 BEST MODEL (Fold {best_fold}):
   • Validation Dice:  {best_dice:.4f}
   • Test Dice (TTA):  {avg_test_dice:.4f}
   • Test IoU (TTA):   {avg_test_iou:.4f}
   • Test Loss:        {avg_test_loss:.4f}

✅ VALIDATION STRATEGY:
   ✓ Proper train/val/test split on GOLD STANDARD data only
   ✓ Pseudo-labels used only for training (not evaluation)
   ✓ K-Fold Cross-Validation for robust performance estimation
   ✓ Test-Time Augmentation for final predictions
   ✓ No data leakage between folds

📝 NOTES:
   • All evaluation metrics are calculated on manually labeled gold standard data
   • Pseudo-labeled data was used to augment training but NOT for evaluation
   • This approach provides a realistic estimate of model performance
   • Cross-validation ensures robustness across different data splits

╚════════════════════════════════════════════════════════════════════════════╝
"""

print(summary_report)

# Save report
with open('final_report.txt', 'w') as f:
    f.write(summary_report)

print("\n✓ Full report saved to 'final_report.txt'")
print("\n" + "="*80)
print("✅ ALL EXPERIMENTS COMPLETE!")
print("="*80)